# Read EEG data and find the start point of the spots

We are going to use [MNE-Python](https://www.martinos.org/mne/stable/index.html), which is a software exploring, visualizing, and analyzing human neurophysiological data: MEG, EEG, sEEG, ECoG, and more.




## Reading data

We are goin to read Brain Vision files, doc is [here](https://www.martinos.org/mne/stable/manual/io.html#brainvision-vhdr-vmrk-eeg).  

In [1]:
import mne.io as io_eeg

#we are going to read an example eeg which is in the folder EEG_example. The method returns a RAW men object
filename = 'eeg_example\\Neuromarketing3850.vhdr'
eeg_object = io_eeg.read_raw_brainvision(filename) #the remaining parameters are left as default

Extracting parameters from eeg_example\Neuromarketing3850.vhdr...
Setting channel info structure...


## Raw Object

The raw object has a lot of methods and some atributes. The most important atributes are:
 * ch_names: Channel names.

 * n_times: Number of time points.

 * times: Time points.

 * info: (dict) [Measurement info](https://www.martinos.org/mne/stable/generated/mne.Info.html#mne.Info). This data structure behaves like a dictionary. It contains all metadata that is available for a recording.

 * preload: (bool) Indicates whether raw data are in memory.

 * verbose : (bool, str, int, or None) If not None, override default verbose level (see mne.verbose() and Logging documentation for more).

In [2]:
print(eeg_object.ch_names)

['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2']


In [3]:
print(eeg_object.n_times)

240140


In [4]:
t = eeg_object.times

In [5]:
print(t.shape)

(240140,)


In [6]:
eeg = eeg_object.get_data()

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
print(eeg.shape)

(31, 240140)


In [8]:
info = eeg_object.info
print(info)

<Info | 16 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, ...
    chs : list | 31 items (EEG: 31)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 140.0 Hz
    meas_date : tuple | 2019-02-21 13:43:42 GMT
    nchan : int | 31
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 500.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    device_info : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    helium_info : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType

In [9]:
# Load Eeg info in a new file called "Data_Info_Eeg"
from TFG_utils import Load_Eeg_Info

Eeg_Data = Load_Eeg_Info(info)    #Data_Info_Eeg --> Eeg_Data

In [10]:
# Get frequency
from TFG_utils import Get_Frequency

sfreq = Get_Frequency(Eeg_Data)    #freq [hz]
print(sfreq)

500


In [11]:
from TFG_utils import take_vmrk_file
 
vmrk_file = take_vmrk_file('eeg_example')

print(vmrk_file)

ImportError: cannot import name 'take_vmrk_file' from 'TFG_utils' (C:\Users\noemi\OneDrive\Escritorio\UNI\TFG\codigo Noemi\TFG-NOEMI\TFG_utils.py)

In [ ]:
# Get Start and end
import os
from TFG_utils import Get_Start_End

start_end = Get_Start_End('eeg_example\\' + vmrk_file)
print(start_end)

In [ ]:
spots_times_sec = [0, 60, 120, 180, 226, 287, 347] # Time at which begins each spot (in sec)
print(spots_times_sec)

In [ ]:
from TFG_utils import spot_samples
spot_samples = spot_samples(start_end,sfreq, spots_times_sec)
print(spot_samples)

In [ ]:
import pylab
import numpy as np
ch=2 #channel to represent
ini= int(60)*int(sfreq) # 1 min of period basal activity, in samples
#print(ini)
i = 0
#print(spot_samples[i])
x=np.arange(spot_samples[i]-ini,spot_samples[i])
plt.plot(x,eeg[ch,spot_samples[i]-ini:spot_samples[i]])
plt.xlabel('Time [sec]')
plt.ylabel('EEG')
 #print(len(spot_samples))
while i < (len(spot_samples)-1):
    x=np.arange(spot_samples[i],spot_samples[i+1])
    plt.plot(x,eeg[ch,spot_samples[i]:spot_samples[i+1]])
    i = i + 1

In [ ]:
# notch filter #

from scipy import signal
import numpy as np

fs= int(sfreq) # Hz, sampling frequency

t = eeg_object.times #generate temporal axes

## Design notch filter

f0 = 50  # Frequency to be removed from signal (Hz)
Q = 30.0  # Quality factor

b,a = signal.iirnotch(f0,Q,fs)
eeg_ch1_notch = signal.filtfilt(b, a, eeg[10,0:len(t)])
plt.figure()
plt.plot(eeg_ch1_notch[0:len(t)])
plt.xlabel('Time [sec]')
plt.ylabel('EEG')

i = 0

while i < (len(spot_samples)-1):
    x=np.arange(spot_samples[i],spot_samples[i+1])
    plt.plot(x,eeg_ch1_notch[spot_samples[i]:spot_samples[i+1]])
    i = i + 1

# Frequency response
freq, h = signal.freqz(b, a, fs=fs)
# Plot        
fig, ax = plt.subplots(2, 1, figsize=(8, 6))
ax[0].plot(freq, 20*np.log10(abs(h)), color='blue')
ax[0].set_title("Frequency Response")
ax[0].set_ylabel("Amplitude (dB)", color='blue')
ax[0].set_xlim([0, 100])
ax[0].set_ylim([-25, 10])
ax[0].grid()
ax[1].plot(freq, np.unwrap(np.angle(h))*180/np.pi, color='green')
ax[1].set_ylabel("Angle (degrees)", color='green')
ax[1].set_xlabel("Frequency (Hz)")
ax[1].set_xlim([0, 100])
ax[1].set_yticks([-90, -60, -30, 0, 30, 60, 90])
ax[1].set_ylim([-90, 90])
ax[1].grid()
plt.show()


In [ ]:
## Design band pass filter            
f1= 2
f2= 30
numtaps=64
b=signal.firwin(numtaps, [f1, f2], pass_zero=False,fs=fs)
eeg_ch1_bp = signal.filtfilt(b, 1, eeg_ch1_notch)
plt.figure()
plt.plot(eeg_ch1_bp[0:len(t)])
plt.xlabel('Time [sec]')
plt.ylabel('EEG')

### plot -> noth + band pass filter
i = 0

while i < (len(spot_samples)-1):
    x=np.arange(spot_samples[i],spot_samples[i+1])
    plt.plot(x,eeg_ch1_bp[spot_samples[i]:spot_samples[i+1]])
    i = i + 1

# Frequency response
freq, h = signal.freqz(b, a, fs=fs)
# Plot
fig, ax = plt.subplots(2, 1, figsize=(8, 6))
ax[0].plot(freq, 20*np.log10(abs(h)), color='blue')
ax[0].set_title("Frequency Response")
ax[0].set_ylabel("Amplitude (dB)", color='blue')
ax[0].set_xlim([0, 100])
ax[0].set_ylim([-50, 50])
ax[0].grid()
ax[1].plot(freq, np.unwrap(np.angle(h))*180/np.pi, color='green')
ax[1].set_ylabel("Angle (degrees)", color='green')
ax[1].set_xlabel("Frequency (Hz)")
ax[1].set_xlim([0, 100])
#ax[1].set_yticks([-90, -60, -30, 0, 30, 60, 90])
#ax[1].set_ylim([-4000, 50])
ax[1].grid()
plt.show()

In [ ]:
### band pass filter     VS    notch - band pass filter ###

## Design band pass filter            
f1= 2
f2= 30
numtaps=64
b=signal.firwin(numtaps, [f1, f2], pass_zero=False,fs=fs)
eeg_ch1_bp = signal.filtfilt(b, 1,  eeg[10,0:len(t)])
eeg_ch1_bp_notch = signal.filtfilt(b, 1, eeg_ch1_notch)
## graficas individuales Notch + bp     y    bp
plt.figure()
plt.plot(eeg_ch1_bp_notch[0:len(t)])
plt.xlabel('Time [sec]')
plt.ylabel('EEG')
plt.figure()
plt.xlabel('Time [sec]')
plt.ylabel('EEG')
plt.plot(eeg_ch1_bp[0:len(t)], color = 'orange')

### grafica superpuesta para comparar ambos resultados
plt.figure()
plt.plot(eeg_ch1_bp_notch[0:len(t)])
plt.xlabel('Time [sec]')
plt.ylabel('EEG')
plt.plot(eeg_ch1_bp[5000:len(t)+5000], color = 'orange')





In [ ]:

import numpy as np
from sklearn.decomposition import PCA

plt.figure()
i=0
eeg_channels_array = [] #  empty regular list
while i < 31:
    ##  notch    No lo aplicamos
    ##  bp
    f1= 2
    f2= 30
    numtaps=64
    b=signal.firwin(numtaps, [f1, f2], pass_zero=False,fs=int(sfreq))
    eeg_chi_bp = signal.filtfilt(b, 1, eeg[i,0:len(t)])

    eeg_channel = eeg_chi_bp # for instance
    eeg_channels_array.append(eeg_channel)
    i += 1
    
np_channels_array = np.array(eeg_channels_array)
print(np_channels_array.shape)

pca = PCA(n_components=4)      
principalComponents = pca.fit(np_channels_array)

plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))      
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

components = pca.transform(np_channels_array)
filtered_channels_array = pca.inverse_transform(components)

plt.figure()
plt.plot(np_channels_array[10, 0:len(t)])
plt.xlabel('Time [sec]')
plt.ylabel('EEG')

plt.figure()
plt.plot(filtered_channels_array[2, 0:len(t)], color = "brown")
plt.xlabel('Time [sec]')
plt.ylabel('EEG')








In [ ]:
import numpy as np
#######  CANALES DE INTERES : [F3, F7, F4, F8]

eeg_interest_channels = np.array([filtered_channels_array[2], filtered_channels_array[3], filtered_channels_array[28], filtered_channels_array[29]])

plt.plot(filtered_channels_array[2])

i=0
while i < 4:
    plt.figure()
    plt.plot(eeg_interest_channels[i, spot_samples[2]: spot_samples[3]])    # distintos canales señal anterior al 1er anuncio
    plt.xlabel('Time [sec]')
    plt.ylabel('EEG')
    if i == 0:
        fig.suptitle("Canal 'F3'")
    i += 1
    
########## Anuncios + prev #########
Prev = []
Anuncio1 = []
Anuncio2 = []
Anuncio3 = []
Anuncio4 = []
Anuncio5 = []
Anuncio6 = []

j = 0
while j < 4:
    i = 0
    while i < (len(spot_samples)-1):    
        x=np.arange(spot_samples[i],spot_samples[i+1])
        if i == 0:
            Prev.append(eeg_interest_channels[j, 0:spot_samples[i]])
            Anuncio1.append(eeg_interest_channels[j, spot_samples[i]:spot_samples[i+1]])
        elif i == 1:
            Anuncio2.append(eeg_interest_channels[j, spot_samples[i]:spot_samples[i+1]])
        elif i == 2:
            Anuncio3.append(eeg_interest_channels[j, spot_samples[i]:spot_samples[i+1]])
        elif i == 3:
            Anuncio4.append(eeg_interest_channels[j, spot_samples[i]:spot_samples[i+1]])
        elif i == 4:
            Anuncio5.append(eeg_interest_channels[j, spot_samples[i]:spot_samples[i+1]])
        elif i == 5:
            Anuncio6.append(eeg_interest_channels[j, spot_samples[i]:spot_samples[i+1]])
        i += 1
    j += 1
    
np_prev = np.array(Prev)
np_anuncio1 = np.array(Anuncio1)
np_anuncio2 = np.array(Anuncio2)
np_anuncio3 = np.array(Anuncio3)
np_anuncio4 = np.array(Anuncio4)
np_anuncio5 = np.array(Anuncio5)
np_anuncio6 = np.array(Anuncio6) 

np_anuncios_t = [np_prev, np_anuncio1, np_anuncio2, np_anuncio3, np_anuncio4, np_anuncio5, np_anuncio6]

    

In [ ]:
plt.figure(figsize = (8,4))
plt.plot(eeg[2, 0:len(t/2)])
plt.plot(eeg[3, 0:len(t/2)])
plt.plot(eeg[28, 0:len(t/2)])
plt.plot(eeg[29, 0:len(t/2)])
plt.figure(figsize = (8,4))
for x in eeg_interest_channels:
    plt.plot(x[0:len(t/2)])
    plt.xlabel('Time [sec]')
    plt.ylabel('EEG')


In [ ]:

import scipy
import statistics as stats

potencias_estimadas_prev = []
potencias_estimadas_anuncio1 = []
potencias_estimadas_anuncio2 = []
potencias_estimadas_anuncio3 = []
potencias_estimadas_anuncio4 = []
potencias_estimadas_anuncio5 = []
potencias_estimadas_anuncio6 = []

#Nsim = 50

j=0
while j < 7:   ## anuncios + prev
    i = 0
    while i < 4:    # canales
        x = np_anuncios_t[j][i, 0:30000]
        f2, Pxx3 =scipy.signal.welch(x, fs = int(sfreq), window='hamming', nperseg=128, nfft=1024)
        l=0
        alpha_frec = []
        while l < (len(f2)):
            if f2[l] >= 8:
                if f2[l] <= 12: 
                    alpha_frec.append(l)  
            l += 1
        
        sumatorio_pxx3 = 0
        for k in Pxx3:
            sumatorio_pxx3 += k
        
        pxx3_rel = []
        for r in Pxx3:
            pxx3_rel.append(r/sumatorio_pxx3) 
       
        potencia_estimada = 0
        for x in alpha_frec:
             potencia_estimada += pxx3_rel[x]
        potencia_estimada_rel = potencia_estimada
        if j == 3 :
            plt.figure()
            plt.plot(f2[0:50], pxx3_rel[0:50])
            plt.xlabel('Frecuencia [Hz]')
            plt.ylabel('Potencia espectral estimada' + str(i))
        
        if j == 0:
            potencias_estimadas_prev.append(potencia_estimada_rel)
        elif j == 1:
            potencias_estimadas_anuncio1.append(potencia_estimada_rel)
        elif j == 2:
            potencias_estimadas_anuncio2.append(potencia_estimada_rel)
        elif j ==3:
            potencias_estimadas_anuncio3.append(potencia_estimada_rel)
        elif j == 4:
            potencias_estimadas_anuncio4.append(potencia_estimada_rel)
        elif j == 5:
            potencias_estimadas_anuncio5.append(potencia_estimada_rel)
        elif j ==6:
            potencias_estimadas_anuncio6.append(potencia_estimada_rel)
        i += 1
    j += 1

labels = ["Actividad normal ", "Anuncio 1", "Anuncio 2", "Anuncio 3", "Anuncio 4", "Anuncio 5", "Anuncio 6"]
potencias_estimadas_f3 = [potencias_estimadas_prev[0], potencias_estimadas_anuncio1[0], potencias_estimadas_anuncio2[0], potencias_estimadas_anuncio3[0], potencias_estimadas_anuncio4[0], potencias_estimadas_anuncio5[0], potencias_estimadas_anuncio6[0]]
potencias_estimadas_f7 = [potencias_estimadas_prev[1], potencias_estimadas_anuncio1[1], potencias_estimadas_anuncio2[1], potencias_estimadas_anuncio3[1], potencias_estimadas_anuncio4[1], potencias_estimadas_anuncio5[1], potencias_estimadas_anuncio6[1]]
potencias_estimadas_f4 = [potencias_estimadas_prev[2], potencias_estimadas_anuncio1[2], potencias_estimadas_anuncio2[2], potencias_estimadas_anuncio3[2], potencias_estimadas_anuncio4[2], potencias_estimadas_anuncio5[2], potencias_estimadas_anuncio6[2]]
potencias_estimadas_f8 = [potencias_estimadas_prev[3], potencias_estimadas_anuncio1[3], potencias_estimadas_anuncio2[3], potencias_estimadas_anuncio3[3], potencias_estimadas_anuncio4[3], potencias_estimadas_anuncio5[3], potencias_estimadas_anuncio6[3]]

x = np.arange(len(labels))  # the label locations
width = 0.17  # the width of the bars


fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - 0.2, potencias_estimadas_f3, width, label='F3')
rects2 = ax.bar(x , potencias_estimadas_f7, width, label='F7')
rects1 = ax.bar(x + 0.2, potencias_estimadas_f4, width, label='F4')
rects2 = ax.bar(x + 0.4, potencias_estimadas_f8, width, label='F8')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Potencia estimada')

ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()









In [ ]:
######  AWI   #####
#AW = GFPa_right - GFPa_left

GFPa_left_prev = (potencias_estimadas_f3[0]+potencias_estimadas_f7[0])/2
GFPa_right_prev = (potencias_estimadas_f4[0]+potencias_estimadas_f8[0])/2

GFPa_left_anuncio1 = (potencias_estimadas_f3[1]+potencias_estimadas_f7[1])/2
GFPa_right_anuncio1 = (potencias_estimadas_f4[1]+potencias_estimadas_f8[1])/2

GFPa_left_anuncio2 = (potencias_estimadas_f3[2]+potencias_estimadas_f7[2])/2
GFPa_right_anuncio2 = (potencias_estimadas_f4[2]+potencias_estimadas_f8[2])/2

GFPa_left_anuncio6 = (potencias_estimadas_f3[3]+potencias_estimadas_f7[3])/2
GFPa_right_anuncio6 = (potencias_estimadas_f4[3]+potencias_estimadas_f8[3])/2

GFPa_left_anuncio3 = (potencias_estimadas_f3[4]+potencias_estimadas_f7[4])/2
GFPa_right_anuncio3 = (potencias_estimadas_f4[4]+potencias_estimadas_f8[4])/2

GFPa_left_anuncio4 = (potencias_estimadas_f3[5]+potencias_estimadas_f7[5])/2
GFPa_right_anuncio4 = (potencias_estimadas_f4[5]+potencias_estimadas_f8[5])/2

GFPa_left_anuncio5 = (potencias_estimadas_f3[6]+potencias_estimadas_f7[6])/2
GFPa_right_anuncio5 = (potencias_estimadas_f4[6]+potencias_estimadas_f8[6])/2

AW_prev = GFPa_right_prev - GFPa_left_prev
AW_anuncio1 = GFPa_right_anuncio1 - GFPa_left_anuncio1
AW_anuncio2 = GFPa_right_anuncio2 - GFPa_left_anuncio2
AW_anuncio3 = GFPa_right_anuncio3 - GFPa_left_anuncio3
AW_anuncio4 = GFPa_right_anuncio4 - GFPa_left_anuncio4
AW_anuncio5 = GFPa_right_anuncio5 - GFPa_left_anuncio5
AW_anuncio6 = GFPa_right_anuncio6 - GFPa_left_anuncio6

AW_anuncios = [AW_anuncio1, AW_anuncio2, AW_anuncio3, AW_anuncio4, AW_anuncio5, AW_anuncio6]

fig, ax = plt.subplots(figsize=(7, 4))
x = np.arange(len(AW_anuncios))
plt.plot(AW_anuncios, marker='o', linestyle='--', color='r')
ax.set_ylabel('AW Index')
ax.set_xticklabels(labels)